# 線形計画法/量子アニーリングでスーパーコンピュータ設計
## 背景
スーパーコンピュータおよびHPC(High Performance Computing)クラスタ（以下、HPCシステム）は、CPU、メモリ、アクセラレータ、インターコネクトなど様々な部品から構成される。
システム設計を行う際、予算および電力容量内で、要求される性能（理論演算性能、コア数、シングルスレッド性能、ノード数、アクセラレータの有無など計算対象の分野によって優先度が異なる）最大化するシステム設計が必要となる。人手で全ての細かい部品について入れ替えて検討することは、組み合わせ数が膨大であり、仕様策定の限られた期間内では困難である。
そこで、このHPCシステム設計を最適化問題として捉え、ヒューリスティックな解法により求める性能の最適構成を高速に手に入れることを目的とする。

この手法は、入力するデータを変更することでHPCシステムに限らず、様々な構成を提案することができる。
D-Waveマシンなどと連携した高速な構成提案システムを検討する。


# CPUについて最適化してみる
様々な部品の組み合わせが考えられるが、計算機ノード内の性能や消費電力に大きく影響を与える主要な部品であるCPUの組み合わせを検討してみる。

## CPUデータの取得
### Intel
Processor関係の製品情報の比較ページでデータをエクスポートすることができる。このデータをtsv形式で保存し、pandasで読み込んで利用する。

旧版？( [ark.intel.com](https://ark.intel.com/content/www/us/en/ark/compare.html?productIds=215286,215272,215281,215279,215271,215285,215284,215282,212461,215274,212458,212286,215273,215280,212285,212633,215278,215276,212457,212456,212460,212288,217215,212307,212454,212284,212282,212308,212459,217216,212455,212289,212287,215275,215277,215283,215269,215270,204091,204097,204089,205688,204086,204090,204088,205687,204095,204092,204094,204096,205683,204087,205684) )のサイトだと定価ベースのデータも取得できる。エクスポートできるデータはHTML形式なので -> tsv -> utf-8への変換を行う必要がある。


新版( [www.intel.com](https://www.intel.com/content/www/us/en/products/compare.html?productIds=199344,197100,197098,193390,193553,193394,193389,193385,193384,193382,199349,215286,212285,212286,212456,212457,212458,212460,212461,212633,215271,215272,215273,215274,215276,215278,215279,215280,215281,215282,215284,215285,204091,204088,204086,204090,204095,205687,217216,212282,212284,212287,212288,212289,212307,212454,212455,212459,217215,212308,205688,204089,204097,205683,204087,204092,204094,204096,205684,195437,195434,194146,194145,192465,192467,192470,192472,192474,192475,192478,192480,192481,192482,192476,120497,120499,120500,120501,120503,120504,120506,120508,123543,123687,125056,120496) )のサイトだと、ログインすると現在価格が表示される（販売されていないものはnanになる）そのままCSVをエクスポートできる。


**今回は新版のサイトから取得したデータを使用する。**

価格データは、手動でintelのspecシートからrecommended priceを取得し、csvファイルに追加した。
なお、Processor number 9242, 9282, 9221, 9222 は価格データがなかったため、当時の予想価格をもとに適当に補完した値としている。

### データの読み込み

In [ ]:
import pandas as pd
# データ読み込み

## intel
df_intel = pd.read_csv('data/Intel_UPE_ComparisonChart_2022_04_15_price.csv', skiprows=1, header=3, index_col=0)


In [ ]:
# intel データの確認
df_intel

In [ ]:
import re

# 使用するデータの整形
# 計算用にnumpy arrayに変換する

#性能比較に使いやすいデータの抽出
df_intel_subset = df_intel.loc[['Total Cores', 'Total Threads', 'Max Turbo Frequency', 'Processor Base Frequency', 'Cache', 'TDP', 'Max Memory Size (dependent on memory type)', 'Maximum Memory Speed', 'Scalability', 'Max # of PCI Express Lanes', 'Recommended Customer Price']]

# Total Coresからndarray作成
totalCores = df_intel_subset.loc['Total Cores'].astype(float)

#print(type(totalCores))
#print(totalCores)

# Total Threadsからndarray作成

totalThreads = df_intel_subset.loc['Total Threads'].astype(float)



# Max Turbo Frequencyから単位[GHz]を除外したndarray作成
# 単位なしでGHzとして扱う
tmp = df_intel_subset.loc['Max Turbo Frequency'].str.split(expand=True)
tmp1 = tmp[0].astype('float').mul(tmp[1].map({'MHz': 0.001, 'GHz': 1}))
maxTurboFreq = tmp1.values

#print(type(maxTurboFreq))
#print(maxTurboFreq)


# Processor Base Frequancyから単位[GHz]を除外したndarray作成
# 単位なしでGHzとして扱う
tmp = df_intel_subset.loc['Processor Base Frequency'].str.split(expand=True)
tmp1 = tmp[0].astype('float').mul(tmp[1].map({'MHz': 0.001, 'GHz': 1}))
procBaseFreq = tmp1.values

#print(type(procBaseFreq))
#print(procBaseFreq)


# Cacheから単位[MB]を除外したndarray作成
tmp = df_intel_subset.loc['Cache'].str.split(expand=True)
tmp1 = tmp[0].astype('float').mul(tmp[1].map({'KB': 0.001, 'MB': 1}))
cache = tmp1.values

#print(cache.shape)
#print(cache)


# TDPから単位[W]を除外したndarray作成
tmp = df_intel_subset.loc['TDP'].str.split(expand=True)
tmp1 = tmp[0].astype('float').mul(tmp[1].map({'W': 1}))
tdp = tmp1.values

#print(tdp)


# Max Memory Sizeから単位[GB], [TB]を除外したndarray作成
tmp = df_intel_subset.loc['Max Memory Size (dependent on memory type)'].str.split(expand=True)
tmp1 = tmp[0].astype('float').mul(tmp[1].map({'GB': 1, 'TB': 1024}))
tmp2 = tmp1.fillna(768)     # nanは768 GBとして扱う
maxMemSize = tmp2.values

#print(maxMemSize)


# Max Memory speedから単位[MHz]を除外したndarray作成
tmp = df_intel_subset.loc['Maximum Memory Speed'].str.split(expand=True)
tmp1 = tmp[0].astype('float').mul(tmp[1].map({'MHz': 1, 'GHz': 1024}))
tmp2 = tmp1.fillna(2933)    # nanは2933 MHzとして扱う
maxMemSpeed = tmp2.values

#print(maxMemSpeed)


# Scalabilityからソケット数上限を抽出
# S8S: Scalable 8 Sockets, 4S: 4 Sockets, ... nan: 1S Onlyに置換
tmp = df_intel_subset.loc['Scalability']
tmp1 = tmp.fillna('1S Only')    # nanは1S Onlyとして扱う
scalability = tmp1.map({'S8S': 8, '4S': 4, '2S': 2, '1S Only': 1}).values

#print(scalability)


# PCI Express Lanesからndarray作成
pcieLanes = df_intel_subset.loc['Max # of PCI Express Lanes'].astype(float)

#print(type(pcieLanes))
#print(pcieLanes)


# Recommended Customer Priceからndarray作成
price = df_intel_subset.loc['Recommended Customer Price']

## 文字列 -> 整数に変換
def convert(price):
    splitted = price.split('.')
    nums = [re.sub("\\D", "", s) for s in splitted]
    return int([n for n in nums if n != ''][0])

intPrice = np.zeros(len(price))
for i in range(len(price)):
    intPrice[i] = convert(price[i])

#print(intPrice)


### 理論演算性能（FLOPS）算出

HPCの性能指標としてFLOPSがよく用いられるため、その値を算出する。

FLOPS = (1clockあたりの演算回数) $\times$ (動作周波数) $\times$ (演算Thread数)



In [ ]:
# 性能指標としてFLOPS算出
## Xeon Gold系は現在1 clock あたり16 FLOPS
op = 16.0

# CPUデータ長取得
M = len(df_intel_subset.columns)

flops = np.zeros(M)

for i in range(M):
    flops[i] = op * maxTurboFreq[i] * totalThreads[i]
    

#print(flops)    # 単位はGFLOPS


### AMD
現状、Intelのみでテスト中。


プロセッサー仕様のWebページから、csvデータを取得できる。このデータをpandasで読み込んで利用する。

https://www.amd.com/ja/products/specifications/processors

In [ ]:
#import pandas as pd
# データ読み込み

## AMD
#df_amd = pd.read_csv('data/AMD_EPYC_Comparison_2022_04_13.csv', header=0, index_col=0)

# AMD データの確認
#df_amd

# GPUについて検討

## GPUデータの取得

HPC分野では、2010年頃からGPGPUが広く普及しており、スパコンにGPUなどアクセラレータを搭載することが一般的となっている。
GPUを搭載したシステムを扱えるようGPU情報も読み込む。

### NVIDIA
NVIDIAのHPC向けGPUの最新モデルはH100である。下記で情報を閲覧できる。

https://www.nvidia.com/ja-jp/data-center/h100/

In [ ]:
df_nvidia = pd.read_csv('data/NVIDIA_GPU_20220419.csv', header=0, index_col=0)

df_nvidia

# 検討中

### AMD
現状、NVIDIAのみでテスト中。

# 線形計画法で解くアイディア

## 線形計画法

### 問題設定
まずは1種類のCPUで1つのシステムの性能を最大化することを目指す。
この時、電力の制約を満たすこと、ノードあたりのCPUコア数の制約条件を満足すること。

### 定数定義:
- $ P_i  $ :CPU iの演算性能の指標として、FLOPSを使う。
- $ C_i $ :CPU iのCache容量
- $ E_i $ :CPU iの消費電力
- $ E_{max} $ :システム最大消費電力
- $ T_i $ :CPU iのコア数/Thread数
- $ T_{sys} $ :CPUあたりのコア数 ノードのコア数をn以上やn以下を指定する場合に使用する
- $ S_i $ :CPUの対応ソケット数
- $ S_{sys} $ :システムの対応ソケット数 ノードあたりいくつのCPUを想定しているか
- $ V_i $ :CPUの価格、ここではrecommended priceを使う
- $ V_{sys} $ :システムにおけるCPUに割り当てる予算の上限

### 決定変数:
- $ x_i $ その種類の数量(整数) $x_i = 0, 1, 2, ...$
- $ z_i $ CPUの種類、採用/採用しない {0 , 1} システム内のCPUを1種類に制限するため使う
- $ y_i = x_i z_i $ 積による表現を目的関数に用いる。

($i = 1,2, ... , N$ CPUの種類)


### 目的関数 (objective):
- 性能の最大化

$$ \max \sum_i^N P_i y_i $$

- キャッシュ容量最大化

$$ \max \sum_i^N C_i y_i $$


### 制約条件 (subject to):
- 電力は設備の最大消費電力以下

$$ \sum_i^N E_i y_i \leq E_{max} $$

- CPUあたりのコア数 <= コア数制限

$$ \sum_i^N T_i z_i \leq T_{sys} $$

- CPUあたりのスケーラビリティ >= ノードあたりのソケット数

$$ \sum_i^N S_i z_i \geq S_{sys} $$

- システムは1種類のCPUで構成する

$$ \sum_i^N z_i = 1 $$

- システムのCPU合計費用は予算以下とする

$$ \sum_i^N V_i y_i = V_{sys} $$ 


In [ ]:
import numpy as np
import pulp


l = 0
u = 10000

# CPUに割り当てる最大消費電力
E_sys = 250000    # 150kW

# CPUあたりの最大コア数
T_sys = 18

# ノードあたりのソケット数
S_sys = 2

# システムのCPUの予算 単位$
V_sys = 4000000


# CPUの種類数取得
M = len(df_intel_subset.columns)

# 決定変数定義
x = {(i):pulp.LpVariable('x%d'%(i), cat="Integer") for i in range(M)}
y = {(i):pulp.LpVariable('y%d'%(i), cat="Integer") for i in range(M)}
z = {(i):pulp.LpVariable('z%d'%(i), cat="Binary") for i in range(M)}

# 最適化モデルの定義
m = pulp.LpProblem(sense=pulp.LpMaximize)


# Objective: -> maximize(性能 & キャッシュ容量)
## maximize 性能=Core数 x 周波数
objective = pulp.lpSum( y[i] * flops[i] for i in range(M) )

## maximize cache容量
objective += pulp.lpSum( y[i] * cache[i] for i in range(M) )

m += objective

# Subject to:
## 電力上限はE_sys以下
m += pulp.lpSum( y[i] * tdp[i] for i in range(M) ) <= E_sys

## CPUあたりのコア数はT_sys以下
m += pulp.lpSum( z[i] * totalCores[i] for i in range(M) ) <= T_sys

## ノードあたりのソケット数はS_sys以上
m += pulp.lpSum( z[i] * scalability[i] for i in range(M) ) >= S_sys

## システムのCPUは1種類のみ
m += pulp.lpSum( z[i] for i in range(M) ) == 1

## 予算内とする
m += pulp.lpSum( y[i] * intPrice[i] for i in range(M) ) <= V_sys

## y = zxの制約条件
for i in range(M):
    m += l * z[i] <= y[i]
    m += u * z[i] >= y[i]

for i in range(M):
    m += x[i] - u * (1 - z[i]) <= y[i]
    m += x[i] - l >= y[i]


# ソルバー設定
solver = pulp.PULP_CBC_CMD()
#solver = pulp.GUROBI_CMD()

m.solve(solver)

# ソルバー起動
m.solve(solver)
        
# 実行可能解が存在したかを表示
print(pulp.LpStatus[m.status])

# 目的関数の計算値
print(pulp.value(m.objective))

# 結果の表示
for k,x in y.items():
    if pulp.value(x) > 0:
        print("Total FLOPS : {} [GFLOPS]".format(pulp.value(x)*flops[k]))
        print("Processor model : {}".format(df_intel.columns[k]))
        print("Number of Processors : {}".format(pulp.value(x)))
        print("Total CPU Price : $ {}".format(pulp.value(x)*intPrice[k]))
        print("Total TDP : {} [W]".format(pulp.value(x)*tdp[k]))
        


# 量子アニーリングで解くアイディア
## Discrete Quadratic Modelsが使えそう
これから考える。